**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape, BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)
        # print('act', a)
        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

epsilon represents the probability to go in a random direction. We have a probability epsilon to go in a random direction and a probability A-epsilon to go in a direction considered optimal by the policiy lerned through the past actions taken. the function learned_act returns the "optimal" direction to go.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat -> RAT
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=401 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array position positions the rat on the board and where he sees the board. The array board represents the state of the game boards, the bonuses and maluses, etc...

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
#         pass
        return np.random.randint(0, 4, 1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
         # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

#             # Apply the reinforcement strategy
#             loss = agent.reinforce(prev_state, state,  action, reward, game_over)

#         agent.save()
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/13.0. Average score (-1.0)
Win/lose count 10.0/17.0. Average score (-4.0)
Win/lose count 8.0/11.0. Average score (-3.6666666666666665)
Win/lose count 5.0/14.0. Average score (-5.0)
Win/lose count 6.0/10.0. Average score (-4.8)
Win/lose count 6.5/9.0. Average score (-4.416666666666667)
Win/lose count 10.0/10.0. Average score (-3.7857142857142856)
Win/lose count 10.0/19.0. Average score (-4.4375)
Win/lose count 6.5/11.0. Average score (-4.444444444444445)
Win/lose count 12.0/11.0. Average score (-3.9)
Win/lose count 12.0/23.0. Average score (-4.545454545454546)
Win/lose count 13.0/22.0. Average score (-4.916666666666667)
Win/lose count 6.5/12.0. Average score (-4.961538461538462)
Win/lose count 8.5/7.0. Average score (-4.5)
Win/lose count 9.5/11.0. Average score (-4.3)
Win/lose count 8.5/8.0. Average score (-4.0)
Win/lose count 10.0/6.0. Average score (-3.5294117647058822)
Win/lose count 8.0/16.0. Average score (-3.7777777777777777)
Win/lose count 11.5/10.0. Average s

In [9]:
HTML(display_videos('random0.mp4'))

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [32]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        self.memory = self.memory[max(len(self.memory)- self.max_memory, 0):]
#         print('REMEMBER')
#         print(max(len(m)- self.max_memory, 0))

    def random_access(self):
        rn = np.random.randint(0, max(len(self.memory), 1))
#         print('MEMORY')
#         print(max(len(self.memory), 1))
#         print(rn)
        return self.memory[rn]

***
The pipeline we will use for training is given below:

In [33]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 100 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [34]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        # Initialize memory, not to get an empty one?
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
#         rewards_ = [0, 0, 0, 0]
#         for action_ in range(0, 4):
#             input_state_ = np.
#             rewards[action_] = self.model.predict(input_state_)
        # print('s', s.reshape(1, 5, 5, 2))
        rewards = self.model.predict(s.reshape(1, 5, 5, self.n_state))[0]
        # print('rew', rewards)
        rew = np.argmax(rewards)
        # print('action', action[0])
        return rew

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
#         print('REINFORCEMENT')
#         print('STATE')
#         print(s_.shape)
#         print(s_)
#         print('REWARD')
#         print(r_.shape)
#         print(r_)
#         print('ACTION')
#         print(a_)
#         print(a_.shape)
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            [s_i, n_s_i, a_i, r_i, game_over_i] = self.memory.random_access()
            # print('re memory', r_i)
            input_states[i] = s_i
            target_q[i] = self.model.predict(s_i.reshape(1, 5, 5, self.n_state))[0]
            if game_over_:
                ######## FILL IN
                target_q[i][a_i] = r_i
            else:
                ######## FILL IN
                # target = self.model.predict(s_i.reshape(1, 5, 5, self.n_state))[0]
                target_ns = self.model.predict(n_s_i.reshape(1, 5, 5, self.n_state))[0]
                # print('model_pred', target, target_ns)
                target_q[i][a_i] = (r_i + self.discount * np.max(target_ns))
                #target_q[i] = target
            
            # print('target :', target_q[i])
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        
        model = Sequential()
        model.add(Dense(20, input_shape=(5, 5, 2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(10, activation='relu'))
        model.add(Dense(4, activation='relu'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

Exception ignored in: <bound method ScopedTFStatus.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFStatus object at 0x0000019602506E48>>
Traceback (most recent call last):
  File "C:\Users\Vincent\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\c_api_util.py", line 39, in __del__
    c_api.TF_DeleteStatus(self.status)
AttributeError: 'ScopedTFStatus' object has no attribute 'status'


In [36]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train%s.mp4' % (epochs_train-1)))

Epoch 000/401 | Loss 0.0253 | Win/lose count 11.5/9.0 (2.5)
Epoch 001/401 | Loss 0.0265 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/401 | Loss 0.0117 | Win/lose count 6.0/6.0 (0.0)
Epoch 003/401 | Loss 0.0038 | Win/lose count 1.5/6.0 (-4.5)
Epoch 004/401 | Loss 0.0353 | Win/lose count 4.0/5.0 (-1.0)
Epoch 005/401 | Loss 0.0305 | Win/lose count 2.5/2.0 (0.5)
Epoch 006/401 | Loss 0.0160 | Win/lose count 6.0/4.0 (2.0)
Epoch 007/401 | Loss 0.0182 | Win/lose count 6.0/3.0 (3.0)
Epoch 008/401 | Loss 0.0208 | Win/lose count 8.5/4.0 (4.5)
Epoch 009/401 | Loss 0.0441 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/401 | Loss 1.9027 | Win/lose count 3.5/4.0 (-0.5)
Epoch 011/401 | Loss 1.9014 | Win/lose count 7.0/3.0 (4.0)
Epoch 012/401 | Loss 1.9739 | Win/lose count 5.5/7.0 (-1.5)
Epoch 013/401 | Loss 1.8748 | Win/lose count 11.0/5.0 (6.0)
Epoch 014/401 | Loss 1.6782 | Win/lose count 9.0/3.0 (6.0)
Epoch 015/401 | Loss 1.8031 | Win/lose count 9.0/1.0 (8.0)
Epoch 016/401 | Loss 1.8077 | Win/lose count 14.0

Epoch 137/401 | Loss 1.7020 | Win/lose count 16.0/3.0 (13.0)
Epoch 138/401 | Loss 1.7374 | Win/lose count 25.0/5.0 (20.0)
Epoch 139/401 | Loss 1.7378 | Win/lose count 18.0/1.0 (17.0)
Epoch 140/401 | Loss 1.7493 | Win/lose count 25.5/8.0 (17.5)
Epoch 141/401 | Loss 1.7604 | Win/lose count 11.5/4.0 (7.5)
Epoch 142/401 | Loss 1.7354 | Win/lose count 17.0/3.0 (14.0)
Epoch 143/401 | Loss 1.7493 | Win/lose count 18.5/4.0 (14.5)
Epoch 144/401 | Loss 1.7476 | Win/lose count 22.5/4.0 (18.5)
Epoch 145/401 | Loss 1.7361 | Win/lose count 15.5/3.0 (12.5)
Epoch 146/401 | Loss 1.7432 | Win/lose count 18.0/3.0 (15.0)
Epoch 147/401 | Loss 1.7838 | Win/lose count 20.5/3.0 (17.5)
Epoch 148/401 | Loss 1.7439 | Win/lose count 4.5/4.0 (0.5)
Epoch 149/401 | Loss 1.7339 | Win/lose count 19.0/4.0 (15.0)
Epoch 150/401 | Loss 1.7822 | Win/lose count 20.5/5.0 (15.5)
Epoch 151/401 | Loss 1.7546 | Win/lose count 22.5/3.0 (19.5)
Epoch 152/401 | Loss 1.7452 | Win/lose count 24.0/1.0 (23.0)
Epoch 153/401 | Loss 1.7307

Epoch 272/401 | Loss 1.8094 | Win/lose count 23.0/0 (23.0)
Epoch 273/401 | Loss 1.7797 | Win/lose count 31.5/5.0 (26.5)
Epoch 274/401 | Loss 1.8373 | Win/lose count 19.5/4.0 (15.5)
Epoch 275/401 | Loss 1.7621 | Win/lose count 9.5/1.0 (8.5)
Epoch 276/401 | Loss 1.7501 | Win/lose count 19.0/1.0 (18.0)
Epoch 277/401 | Loss 1.7903 | Win/lose count 19.5/2.0 (17.5)
Epoch 278/401 | Loss 1.8093 | Win/lose count 7.5/0 (7.5)
Epoch 279/401 | Loss 1.7977 | Win/lose count 26.5/5.0 (21.5)
Epoch 280/401 | Loss 1.7766 | Win/lose count 26.0/5.0 (21.0)
Epoch 281/401 | Loss 1.7793 | Win/lose count 22.0/5.0 (17.0)
Epoch 282/401 | Loss 1.7769 | Win/lose count 27.0/5.0 (22.0)
Epoch 283/401 | Loss 1.7618 | Win/lose count 27.5/1.0 (26.5)
Epoch 284/401 | Loss 1.7432 | Win/lose count 20.0/4.0 (16.0)
Epoch 285/401 | Loss 1.6971 | Win/lose count 16.0/2.0 (14.0)
Epoch 286/401 | Loss 1.8019 | Win/lose count 18.5/1.0 (17.5)
Epoch 287/401 | Loss 1.7725 | Win/lose count 21.5/4.0 (17.5)
Epoch 288/401 | Loss 1.7738 | Wi

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [37]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, 8, 8, border_mode='same', input_shape=(5,5,self.n_state)))
        model.add(Activation('relu'))
        model.add(Conv2D(16, 4, 4, border_mode='same'))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self.n_action))
        
        model.compile('adam', "mse")
        self.model = model

In [39]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train%s.mp4' % (epochs_train-1)))

C:\Users\Vincent\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(5, 5, 2), padding="same")`
  import sys
C:\Users\Vincent\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (4, 4), padding="same")`
  if __name__ == '__main__':


Epoch 000/401 | Loss 1.5927 | Win/lose count 3.5/5.0 (-1.5)
Epoch 001/401 | Loss 1.1646 | Win/lose count 9.0/15.0 (-6.0)
Epoch 002/401 | Loss 1.7595 | Win/lose count 5.0/4.0 (1.0)
Epoch 003/401 | Loss 1.5560 | Win/lose count 8.5/4.0 (4.5)
Epoch 004/401 | Loss 1.7915 | Win/lose count 6.5/2.0 (4.5)
Epoch 005/401 | Loss 1.7141 | Win/lose count 8.0/2.0 (6.0)
Epoch 006/401 | Loss 1.7585 | Win/lose count 8.5/2.0 (6.5)
Epoch 007/401 | Loss 1.8751 | Win/lose count 4.5/4.0 (0.5)
Epoch 008/401 | Loss 1.8375 | Win/lose count 9.0/2.0 (7.0)
Epoch 009/401 | Loss 1.8542 | Win/lose count 6.0/3.0 (3.0)
Epoch 010/401 | Loss 1.7742 | Win/lose count 12.0/5.0 (7.0)
Epoch 011/401 | Loss 1.9422 | Win/lose count 14.0/6.0 (8.0)
Epoch 012/401 | Loss 1.9723 | Win/lose count 14.5/4.0 (10.5)
Epoch 013/401 | Loss 1.9299 | Win/lose count 8.5/4.0 (4.5)
Epoch 014/401 | Loss 1.9512 | Win/lose count 23.0/5.0 (18.0)
Epoch 015/401 | Loss 1.9696 | Win/lose count 9.0/2.0 (7.0)
Epoch 016/401 | Loss 1.8373 | Win/lose count 10

Epoch 137/401 | Loss 1.8466 | Win/lose count 4.0/2.0 (2.0)
Epoch 138/401 | Loss 1.8807 | Win/lose count 10.0/3.0 (7.0)
Epoch 139/401 | Loss 1.7027 | Win/lose count 14.0/0 (14.0)
Epoch 140/401 | Loss 1.9174 | Win/lose count 15.5/4.0 (11.5)
Epoch 141/401 | Loss 1.8459 | Win/lose count 10.0/1.0 (9.0)
Epoch 142/401 | Loss 1.8651 | Win/lose count 24.0/3.0 (21.0)
Epoch 143/401 | Loss 1.8714 | Win/lose count 19.5/6.0 (13.5)
Epoch 144/401 | Loss 2.0414 | Win/lose count 19.5/2.0 (17.5)
Epoch 145/401 | Loss 1.8695 | Win/lose count 9.5/1.0 (8.5)
Epoch 146/401 | Loss 1.9789 | Win/lose count 12.5/1.0 (11.5)
Epoch 147/401 | Loss 1.8806 | Win/lose count 11.5/4.0 (7.5)
Epoch 148/401 | Loss 1.8254 | Win/lose count 21.0/3.0 (18.0)
Epoch 149/401 | Loss 1.9636 | Win/lose count 12.0/0 (12.0)
Epoch 150/401 | Loss 1.8902 | Win/lose count 20.0/3.0 (17.0)
Epoch 151/401 | Loss 1.8907 | Win/lose count 20.5/4.0 (16.5)
Epoch 152/401 | Loss 1.8548 | Win/lose count 17.0/5.0 (12.0)
Epoch 153/401 | Loss 1.8399 | Win/l

Epoch 273/401 | Loss 1.8445 | Win/lose count 20.0/3.0 (17.0)
Epoch 274/401 | Loss 1.8769 | Win/lose count 14.0/6.0 (8.0)
Epoch 275/401 | Loss 1.8256 | Win/lose count 9.5/4.0 (5.5)
Epoch 276/401 | Loss 1.7585 | Win/lose count 13.0/5.0 (8.0)
Epoch 277/401 | Loss 1.7540 | Win/lose count 21.0/2.0 (19.0)
Epoch 278/401 | Loss 1.7408 | Win/lose count 19.0/2.0 (17.0)
Epoch 279/401 | Loss 1.7188 | Win/lose count 16.5/5.0 (11.5)
Epoch 280/401 | Loss 1.7121 | Win/lose count 10.5/3.0 (7.5)
Epoch 281/401 | Loss 1.6635 | Win/lose count 7.0/5.0 (2.0)
Epoch 282/401 | Loss 1.7021 | Win/lose count 14.0/5.0 (9.0)
Epoch 283/401 | Loss 1.6465 | Win/lose count 17.0/5.0 (12.0)
Epoch 284/401 | Loss 1.7268 | Win/lose count 19.0/3.0 (16.0)
Epoch 285/401 | Loss 1.7830 | Win/lose count 20.5/3.0 (17.5)
Epoch 286/401 | Loss 1.7346 | Win/lose count 12.5/2.0 (10.5)
Epoch 287/401 | Loss 1.8130 | Win/lose count 17.0/4.0 (13.0)
Epoch 288/401 | Loss 1.6817 | Win/lose count 17.0/6.0 (11.0)
Epoch 289/401 | Loss 1.8505 | Wi

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [40]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

C:\Users\Vincent\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(5, 5, 2), padding="same")`
  import sys
C:\Users\Vincent\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (4, 4), padding="same")`
  if __name__ == '__main__':


Test of the CNN
Win/lose count 20.0/5.0. Average score (15.0)
Win/lose count 19.0/7.0. Average score (13.5)
Win/lose count 13.0/3.0. Average score (12.333333333333334)
Win/lose count 13.0/1.0. Average score (12.25)
Win/lose count 25.0/2.0. Average score (14.4)
Win/lose count 22.5/5.0. Average score (14.916666666666666)
Win/lose count 16.0/0. Average score (15.071428571428571)
Win/lose count 20.5/3.0. Average score (15.375)
Win/lose count 15.5/1.0. Average score (15.277777777777779)
Win/lose count 12.0/1.0. Average score (14.85)
Win/lose count 17.0/4.0. Average score (14.681818181818182)
Win/lose count 13.0/4.0. Average score (14.208333333333334)
Win/lose count 25.5/3.0. Average score (14.846153846153847)
Win/lose count 23.0/6.0. Average score (15.0)
Win/lose count 14.5/3.0. Average score (14.766666666666667)
Win/lose count 17.5/4.0. Average score (14.6875)
Win/lose count 24.0/5.0. Average score (14.941176470588236)
Win/lose count 18.5/4.0. Average score (14.916666666666666)
Win/lose co

In [42]:
HTML(display_videos('cnn_test%s.mp4' % (epochs_test-1)))

In [43]:
HTML(display_videos('fc_test%s.mp4' % (epochs_test-1)))

The algorithm tends to stay stuck between two states. It happens particularly when the rat cannot see any bonus (red case) in his field of vision. Sometimes a random action makes the rat leave these states and unstuck him. Of cours this last case happens more often when the temperature is high.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [44]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    agent.set_epsilon(1)
    for e in range(epoch):
        agent.set_epsilon(agent.epsilon - 1/epoch)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 100 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
  

In [47]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat -> RAT
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [48]:
# Training
epochs_train=801
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore%s.mp4' % (epochs_train-1)))

C:\Users\Vincent\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), input_shape=(5, 5, 3), padding="same")`
  import sys
C:\Users\Vincent\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (4, 4), padding="same")`
  if __name__ == '__main__':


Epoch 000/801 | Loss 0.0356 | Win/lose count 4.5/22.600000000000033 (-18.100000000000033)
Epoch 001/801 | Loss 0.7179 | Win/lose count 12.0/21.20000000000003 (-9.200000000000031)
Epoch 002/801 | Loss 1.7476 | Win/lose count 19.0/18.400000000000016 (0.5999999999999837)
Epoch 003/801 | Loss 1.5894 | Win/lose count 18.0/14.89999999999997 (3.10000000000003)
Epoch 004/801 | Loss 1.5294 | Win/lose count 14.0/19.599999999999994 (-5.599999999999994)
Epoch 005/801 | Loss 1.6261 | Win/lose count 16.5/14.499999999999972 (2.0000000000000284)
Epoch 006/801 | Loss 1.5343 | Win/lose count 15.0/18.799999999999994 (-3.7999999999999936)
Epoch 007/801 | Loss 1.6097 | Win/lose count 21.0/17.099999999999987 (3.900000000000013)
Epoch 008/801 | Loss 1.5773 | Win/lose count 19.5/13.099999999999973 (6.400000000000027)
Epoch 009/801 | Loss 1.4539 | Win/lose count 22.5/14.899999999999967 (7.600000000000033)
Epoch 010/801 | Loss 1.5251 | Win/lose count 10.0/17.499999999999986 (-7.499999999999986)
Epoch 011/801 | 

Epoch 092/801 | Loss 1.6329 | Win/lose count 19.5/14.899999999999967 (4.600000000000033)
Epoch 093/801 | Loss 1.4662 | Win/lose count 16.5/15.199999999999969 (1.300000000000031)
Epoch 094/801 | Loss 1.5126 | Win/lose count 24.5/13.899999999999972 (10.600000000000028)
Epoch 095/801 | Loss 1.4506 | Win/lose count 10.0/20.800000000000036 (-10.800000000000036)
Epoch 096/801 | Loss 1.4942 | Win/lose count 22.5/16.499999999999982 (6.000000000000018)
Epoch 097/801 | Loss 1.4780 | Win/lose count 26.0/15.899999999999977 (10.100000000000023)
Epoch 098/801 | Loss 1.6141 | Win/lose count 29.0/13.999999999999984 (15.000000000000016)
Epoch 099/801 | Loss 1.5064 | Win/lose count 15.0/15.99999999999997 (-0.9999999999999698)
Epoch 100/801 | Loss 1.5736 | Win/lose count 28.5/15.99999999999998 (12.50000000000002)
Epoch 101/801 | Loss 1.6325 | Win/lose count 21.0/12.29999999999998 (8.70000000000002)
Epoch 102/801 | Loss 1.6320 | Win/lose count 21.5/13.09999999999998 (8.40000000000002)
Epoch 103/801 | Loss

Epoch 185/801 | Loss 1.5905 | Win/lose count 27.0/9.89999999999998 (17.10000000000002)
Epoch 186/801 | Loss 1.6162 | Win/lose count 16.0/16.49999999999997 (-0.4999999999999716)
Epoch 187/801 | Loss 1.5689 | Win/lose count 24.0/16.9 (7.100000000000001)
Epoch 188/801 | Loss 1.5958 | Win/lose count 20.0/12.999999999999973 (7.000000000000027)
Epoch 189/801 | Loss 1.6931 | Win/lose count 19.5/12.99999999999998 (6.5000000000000195)
Epoch 190/801 | Loss 1.6456 | Win/lose count 23.0/13.999999999999977 (9.000000000000023)
Epoch 191/801 | Loss 1.6444 | Win/lose count 19.0/18.300000000000004 (0.6999999999999957)
Epoch 192/801 | Loss 1.4913 | Win/lose count 13.0/16.899999999999977 (-3.8999999999999773)
Epoch 193/801 | Loss 1.5744 | Win/lose count 25.5/15.699999999999973 (9.800000000000027)
Epoch 194/801 | Loss 1.3800 | Win/lose count 15.0/16.099999999999966 (-1.099999999999966)
Epoch 195/801 | Loss 1.6451 | Win/lose count 25.5/12.399999999999974 (13.100000000000026)
Epoch 196/801 | Loss 1.6118 | W

Epoch 277/801 | Loss 1.2816 | Win/lose count 22.5/16.199999999999974 (6.300000000000026)
Epoch 278/801 | Loss 1.3606 | Win/lose count 15.0/23.70000000000006 (-8.70000000000006)
Epoch 279/801 | Loss 1.2822 | Win/lose count 16.0/19.299999999999986 (-3.2999999999999865)
Epoch 280/801 | Loss 1.3880 | Win/lose count 20.5/15.599999999999971 (4.900000000000029)
Epoch 281/801 | Loss 1.2755 | Win/lose count 12.5/15.999999999999966 (-3.4999999999999662)
Epoch 282/801 | Loss 1.1849 | Win/lose count 16.0/16.799999999999976 (-0.7999999999999758)
Epoch 283/801 | Loss 1.2520 | Win/lose count 16.5/14.599999999999968 (1.9000000000000323)
Epoch 284/801 | Loss 1.2838 | Win/lose count 17.0/13.999999999999966 (3.0000000000000338)
Epoch 285/801 | Loss 1.2147 | Win/lose count 24.5/15.59999999999997 (8.90000000000003)
Epoch 286/801 | Loss 1.2691 | Win/lose count 17.5/16.899999999999984 (0.6000000000000156)
Epoch 287/801 | Loss 1.3506 | Win/lose count 24.5/16.099999999999977 (8.400000000000023)
Epoch 288/801 |

Epoch 369/801 | Loss 1.1393 | Win/lose count 17.0/14.899999999999967 (2.1000000000000334)
Epoch 370/801 | Loss 1.4192 | Win/lose count 19.5/18.399999999999977 (1.1000000000000227)
Epoch 371/801 | Loss 1.4987 | Win/lose count 20.0/20.30000000000002 (-0.3000000000000185)
Epoch 372/801 | Loss 1.3492 | Win/lose count 13.5/18.09999999999998 (-4.59999999999998)
Epoch 373/801 | Loss 1.3851 | Win/lose count 24.0/17.8 (6.199999999999999)
Epoch 374/801 | Loss 1.1036 | Win/lose count 20.0/13.69999999999997 (6.300000000000029)
Epoch 375/801 | Loss 1.4210 | Win/lose count 12.5/20.50000000000004 (-8.000000000000039)
Epoch 376/801 | Loss 1.2942 | Win/lose count 19.0/16.09999999999997 (2.9000000000000306)
Epoch 377/801 | Loss 1.5232 | Win/lose count 18.0/14.599999999999971 (3.4000000000000288)
Epoch 378/801 | Loss 1.2394 | Win/lose count 15.5/19.40000000000002 (-3.90000000000002)
Epoch 379/801 | Loss 1.5010 | Win/lose count 12.5/20.50000000000004 (-8.000000000000039)
Epoch 380/801 | Loss 1.2376 | Win/

Epoch 462/801 | Loss 1.3962 | Win/lose count 14.5/14.899999999999967 (-0.3999999999999666)
Epoch 463/801 | Loss 1.1495 | Win/lose count 5.5/22.100000000000026 (-16.600000000000026)
Epoch 464/801 | Loss 1.1074 | Win/lose count 19.0/18.49999999999999 (0.5000000000000107)
Epoch 465/801 | Loss 1.2491 | Win/lose count 20.0/14.499999999999972 (5.500000000000028)
Epoch 466/801 | Loss 1.3064 | Win/lose count 16.5/21.200000000000024 (-4.700000000000024)
Epoch 467/801 | Loss 1.1907 | Win/lose count 17.0/21.200000000000053 (-4.200000000000053)
Epoch 468/801 | Loss 1.0611 | Win/lose count 14.0/15.099999999999968 (-1.0999999999999677)
Epoch 469/801 | Loss 1.1546 | Win/lose count 10.0/16.899999999999977 (-6.899999999999977)
Epoch 470/801 | Loss 1.1473 | Win/lose count 20.5/17.79999999999999 (2.70000000000001)
Epoch 471/801 | Loss 1.0057 | Win/lose count 14.5/15.799999999999967 (-1.299999999999967)
Epoch 472/801 | Loss 1.1542 | Win/lose count 18.0/20.70000000000001 (-2.70000000000001)
Epoch 473/801 |

Epoch 555/801 | Loss 0.9864 | Win/lose count 15.0/18.799999999999997 (-3.799999999999997)
Epoch 556/801 | Loss 1.0989 | Win/lose count 11.0/21.40000000000005 (-10.400000000000048)
Epoch 557/801 | Loss 0.9117 | Win/lose count 21.5/17.599999999999987 (3.900000000000013)
Epoch 558/801 | Loss 1.1433 | Win/lose count 8.0/19.40000000000001 (-11.40000000000001)
Epoch 559/801 | Loss 0.8227 | Win/lose count 19.5/17.899999999999995 (1.600000000000005)
Epoch 560/801 | Loss 0.9429 | Win/lose count 11.5/21.200000000000035 (-9.700000000000035)
Epoch 561/801 | Loss 1.0837 | Win/lose count 18.0/14.599999999999971 (3.4000000000000288)
Epoch 562/801 | Loss 1.0633 | Win/lose count 13.0/21.50000000000002 (-8.500000000000021)
Epoch 563/801 | Loss 0.9292 | Win/lose count 10.5/19.80000000000001 (-9.300000000000011)
Epoch 564/801 | Loss 0.8690 | Win/lose count 17.5/21.50000000000004 (-4.000000000000039)
Epoch 565/801 | Loss 0.8816 | Win/lose count 18.5/20.50000000000003 (-2.0000000000000284)
Epoch 566/801 | L

Epoch 647/801 | Loss 1.5701 | Win/lose count 17.5/16.699999999999974 (0.8000000000000256)
Epoch 648/801 | Loss 1.2779 | Win/lose count 22.0/12.899999999999972 (9.100000000000028)
Epoch 649/801 | Loss 1.2459 | Win/lose count 15.0/19.300000000000004 (-4.300000000000004)
Epoch 650/801 | Loss 1.4387 | Win/lose count 10.5/18.2 (-7.699999999999999)
Epoch 651/801 | Loss 1.3025 | Win/lose count 7.0/16.59999999999997 (-9.59999999999997)
Epoch 652/801 | Loss 1.0653 | Win/lose count 9.5/22.200000000000067 (-12.700000000000067)
Epoch 653/801 | Loss 1.1558 | Win/lose count 17.0/20.000000000000036 (-3.0000000000000355)
Epoch 654/801 | Loss 1.1095 | Win/lose count 15.0/14.799999999999963 (0.2000000000000366)
Epoch 655/801 | Loss 0.9151 | Win/lose count 11.5/24.600000000000048 (-13.100000000000048)
Epoch 656/801 | Loss 1.1206 | Win/lose count 8.5/19.40000000000002 (-10.90000000000002)
Epoch 657/801 | Loss 1.1023 | Win/lose count 20.5/21.400000000000013 (-0.9000000000000128)
Epoch 658/801 | Loss 1.5283

Epoch 740/801 | Loss 1.2032 | Win/lose count 26.0/17.400000000000002 (8.599999999999998)
Epoch 741/801 | Loss 1.2355 | Win/lose count 17.0/21.300000000000026 (-4.300000000000026)
Epoch 742/801 | Loss 1.0221 | Win/lose count 17.5/18.6 (-1.1000000000000014)
Epoch 743/801 | Loss 1.2724 | Win/lose count 15.0/20.20000000000002 (-5.200000000000021)
Epoch 744/801 | Loss 1.1679 | Win/lose count 22.5/16.799999999999983 (5.700000000000017)
Epoch 745/801 | Loss 0.9536 | Win/lose count 19.5/18.599999999999987 (0.9000000000000128)
Epoch 746/801 | Loss 1.1732 | Win/lose count 22.0/19.29999999999999 (2.70000000000001)
Epoch 747/801 | Loss 0.9418 | Win/lose count 19.0/21.80000000000006 (-2.800000000000061)
Epoch 748/801 | Loss 1.1339 | Win/lose count 26.5/15.29999999999997 (11.20000000000003)
Epoch 749/801 | Loss 1.2151 | Win/lose count 9.5/20.70000000000003 (-11.200000000000031)
Epoch 750/801 | Loss 1.0718 | Win/lose count 20.5/17.69999999999998 (2.8000000000000185)
Epoch 751/801 | Loss 1.1906 | Win/

In [49]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore%s.mp4' % (epochs_train-1)))

Win/lose count 12.5/4.0. Average score (8.5)
Win/lose count 21.5/3.0. Average score (13.5)
Win/lose count 11.5/5.0. Average score (11.166666666666666)
Win/lose count 22.5/7.0. Average score (12.25)
Win/lose count 9.5/8.0. Average score (10.1)
Win/lose count 18.5/7.0. Average score (10.333333333333334)
Win/lose count 7.5/1.0. Average score (9.785714285714286)
Win/lose count 11.5/11.0. Average score (8.625)
Win/lose count 18.5/9.0. Average score (8.722222222222221)
Win/lose count 15.0/11.0. Average score (8.25)
Win/lose count 9.5/4.0. Average score (8.0)
Win/lose count 17.5/6.0. Average score (8.291666666666666)
Win/lose count 22.0/8.0. Average score (8.73076923076923)
Win/lose count 12.5/11.0. Average score (8.214285714285714)
Win/lose count 18.5/5.0. Average score (8.566666666666666)
Win/lose count 26.5/3.0. Average score (9.5)
Win/lose count 11.5/7.0. Average score (9.205882352941176)
Win/lose count 13.5/7.0. Average score (9.055555555555555)
Win/lose count 6.0/4.0. Average score (8.6

FileNotFoundError: [Errno 2] No such file or directory: 'cnn_test_explore800.mp4'

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***